In [ ]:
import numpy as np
import pandas as pd
from itertools import groupby
import re
import os

In [ ]:
# Change working directory to be in the project folder and print out what is in the data folder
os.chdir('/content/drive/MyDrive/Free time (Aaron Presser): Poverty prediction project (02 14 2022)/')
os.listdir('Output_csv_files')

['food_access_df.csv',
 'food_env_df.csv',
 'merged_df.csv',
 'fastfood_df.csv',
 'fastfood_df_dup.csv',
 'merged_df_10.csv',
 'walmart_df.csv']

In [ ]:
# Read in data
walmart_df = pd.read_csv('Output_csv_files/walmart_df.csv')
food_access_df = pd.read_csv('Output_csv_files/food_access_df.csv')
food_env_df = pd.read_csv('Output_csv_files/food_env_df.csv')
fastfood_df = pd.read_csv('Output_csv_files/fastfood_df.csv')

In [ ]:
# Premerging cleaning
fastfood_df['Zip Code'] = fastfood_df['Zip Code'].apply(lambda x: x.split('-')[0])
fastfood_df['Zip Code'] = fastfood_df['Zip Code'].astype(float)

In [ ]:
# Merge dfs - fill in missing values (that are created by merging) inductively(?)
interm = pd.merge_ordered(walmart_df, food_access_df, on='Zip Code', fill_method='ffill')
interm = pd.merge_ordered(food_env_df, interm, on='Zip Code', fill_method='ffill')
merged_df = pd.merge_ordered(fastfood_df, interm, on='Zip Code', fill_method='ffill')
merged_df.dropna(subset=['Zip Code', 'Restaurant Name'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:321: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Unnamed: 0_y'} in the result is deprecated and will raise a MergeError in a future version.
  result = _merger(left, right)


In [ ]:
# Tidy up merged df
merged_df['Zip Code'] = merged_df['Zip Code'].astype(int)

cols_to_drop = ['Address_x', 'Latitude_x', 'Longitude_x', 'Address_y', 'Latitude_y', 'Longitude_y', 'WhenAdded',\
                'LastUpdated', 'Unnamed: 0_x', 'Unnamed: 0_y', 'FIPS', 'State_x', 'County_x', 'NUMGQTRS',\
                'PrimaryCategories', 'City', 'State_y', 'County_y', 'CensusTract', 'State']
merged_df.drop(columns=cols_to_drop, inplace=True)
merged_df

,Restaurant Name,Zip Code,Catering,Deli,Hot dogs,Mexican,Sandwich,Coffee,Vegetarian,Pizza,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
193,Black Cow Burger Bar,1376,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,631.0,3798.0,74.0,29.0,1.0,18.0,293.0,346.0,268.0,449.0
194,Black Cow Burger Bar,1376,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,715.0,4025.0,27.0,38.0,1.0,7.0,126.0,92.0,122.0,311.0
195,Black Cow Burger Bar,1380,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,459.0,3319.0,21.0,17.0,0.0,16.0,55.0,60.0,70.0,213.0
196,Black Cow Burger Bar,1420,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,330.0,2770.0,149.0,104.0,0.0,14.0,187.0,396.0,15.0,55.0
197,Black Cow Burger Bar,1420,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,306.0,1844.0,41.0,126.0,0.0,6.0,132.0,271.0,24.0,179.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129699,Pizza Mill,99926,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,133.0,145.0,6.0,2.0,10.0,1210.0,87.0,26.0,59.0,164.0
129700,Pizza Mill,99927,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,181.0,1217.0,5.0,6.0,7.0,351.0,99.0,29.0,111.0,154.0
129701,Pizza Mill,99928,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,351.0,3013.0,9.0,33.0,4.0,186.0,239.0,103.0,58.0,116.0
129702,Pizza Mill,99928,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,351.0,3013.0,9.0,33.0,4.0,186.0,239.0,103.0,58.0,116.0


In [ ]:
# Define function that filters rows and cols from df according to how many NAs they have
def col_row_na_filterer(df, col_thresh=None, row_thresh=None):
  """Returns df with columns and rows filtered according to what proportion of NAs they have
  
  Args:
    df - Dataframe
    col_thresh - What proportion of NAs will cause a column to be dropped
    row_thresh - What proportion of NAs will cause a row to be dropped
  
  Returns:
    df - Dataframe with cols and rows dropped
  """
  if row_thresh is not None:
    thresh = len(df.columns) - np.ceil(row_thresh*len(df.columns))
    df.dropna(thresh=thresh, inplace=True)

  cols_to_drop = []
  if col_thresh is not None:
    for col in df.columns:
      nas = df[col].isna()
      df_sub = df[nas]
      if ((len(list(df_sub[col].isna()))/len(list(df[col]))) > col_thresh):
        cols_to_drop.append(col)
        print("Dropped {}...".format(col))

    df.drop(cols_to_drop, axis=1, inplace=True)

  return df

In [ ]:
def what_is_low_access(df: dataframe, distance: int) -> df:
  """Returns a df containing the low-access column that the user defines as low-access
  with other low-access columns dropped
  
  Args:
    df - Dataframe
    distance - What the user defines to be low-access distance
    
  Returns:
    df - Dataframe contining low-access column user wants and other low-access
    columns dropped
  """
  shit = df.filter(regex='^LA', axis=1).columns
  to_drop_interm = shit[~shit.str.endswith(f'{distance}')]
  to_drop = shit[shit.str.contains('and')]
  df.drop(to_drop, axis=1, inplace=True)
  return df

In [ ]:
def reorder_based_off_low_access(df: dataframe, distance: int) -> df:
  """Moves low-access variable to front of df
  
  Args:
    df - Dataframe
    distance - What the user defines to be low-access
    
  Returns:
    df - Dataframe with low-access variable as first column
  """
  low_access = df.columns[df.columns.str.startswith('LA')]
  low_access = low_access[low_access.str.endswith(f'{distance}')]

    # assert len(low_access) == 1

  low_access_df = df[low_access]
  df = df.drop(columns=low_access.tolist())
  return low_access_df.join(df)

In [ ]:
def clean_locations(df: dataframe) -> df:
  """Returns dataframe with similar series values mapped to single replacement_name values
  
  Args:
    df - Dataframe
    
  Returns:
    df - Dataframe
  """
  str_filters = ['Walmart SC', 'Sam', 'Murphy', 'Wm', 'WM', 'wm', 'Walmart;']
  replacement_names = ['SC', 'Sam\'s Club', 'Murphy', 'Walmart', 'Walmart', 'Walmart', 'Walmart']

  for i, j in zip(str_filters, replacement_names):
    boolean_mask = df['Location'].apply(lambda x: x.startswith(i))
    df.loc[boolean_mask, 'Location'] = j

  return df

In [ ]:
def process_contains(df: dataframe, column: series, contains: list, replace: list) -> dataframe:
  """Finds elements of column that contain string-fragments and returns df with 
  columns of binary variables for those string-fragments
  
  Args:
    df - Dataframe
    column - In which column to check for string-fragments
    contains - List of string-fragments that are to be searched for
    replace - List of what to replace those string-fragments with
    
  Returns:
    df - Dataframe with columns of binary variables indicating presence or absence
    of the string-fragments searched for
  """
  if contains is None:
    contains = ['Cater', 'Delic', 'Hot dog', 'Mexican', 'Sandwich', 'Coffee', 'Veg', 'Pizza']
  else: contains = set(df[f"{contains}"])
  
  if replace is None:
    replace = ['Catering', 'Deli', 'Hot dogs', 'Mexican', 'Sandwich', 'Coffee', 'Vegetarian', 'Pizza']
  else: replace = set(df[f"{replace}"])
  
  if column is None:
    column = 'categories'

  variable_dict = {}
  for i, j, k in zip(contains, replace, range(len(contains))):
    boolean_mask = df[f"{column}"].str.contains(i)
    variable_dict[f'{j}'] = boolean_mask
  
  df.drop(columns=[f"{column}"], inplace=True)
  variable_df = pd.DataFrame(variable_dict)
  variable_df.replace({False: 0, True: 1}, inplace=True)

  return df.join(variable_df)

In [ ]:
def main(df):
  df = col_row_na_filterer(df, 0.99, 0.99)
  df = what_is_low_access(df, 10)
  df = reorder_based_off_low_access(df, 10)
  df = clean_locations(df)
  df.reset_index(drop=True, inplace=True)

  restaurant_name = ",".join(df['Restaurant Name']).split(",")
  df['Restaurant Name'].str.get_dummies(",")

  df.drop(columns=['Restaurant Name'], inplace=True)
  df.drop(index=df[df['Location'].apply(lambda x: x.startswith('Walmart '))].index, inplace=True)

  df = process_contains(df, column = 'Location', contains='Location', replace='Location')
  return df

In [ ]:
merged_df = main(merged_df)
merged_df

Dropped LACCESS_BLACK15...
Dropped LACCESS_CHILD10...
Dropped LACCESS_CHILD15...
Dropped LACCESS_CHILD_10_15...
Dropped LACCESS_HHNV10...
Dropped LACCESS_HHNV15...
Dropped LACCESS_HISP15...
Dropped LACCESS_LOWI10...
Dropped LACCESS_LOWI15...
Dropped LACCESS_MULTIR15...
Dropped LACCESS_NHASIAN15...
Dropped LACCESS_NHNA15...
Dropped LACCESS_NHPI15...
Dropped LACCESS_POP10...
Dropped LACCESS_POP15...
Dropped LACCESS_SENIORS10...
Dropped LACCESS_SENIORS15...
Dropped LACCESS_SNAP15...
Dropped LACCESS_WHITE15...
Dropped PCH_LACCESS_HHNV_10_15...
Dropped PCH_LACCESS_LOWI_10_15...
Dropped PCH_LACCESS_POP_10_15...
Dropped PCH_LACCESS_SENIORS_10_15...
Dropped PCT_LACCESS_BLACK15...
Dropped PCT_LACCESS_CHILD10...
Dropped PCT_LACCESS_CHILD15...
Dropped PCT_LACCESS_HHNV10...
Dropped PCT_LACCESS_HHNV15...
Dropped PCT_LACCESS_HISP15...
Dropped PCT_LACCESS_LOWI10...
Dropped PCT_LACCESS_LOWI15...
Dropped PCT_LACCESS_MULTIR15...
Dropped PCT_LACCESS_NHASIAN15...
Dropped PCT_LACCESS_NHNA15...
Dropped PCT_

,LATracts10,Restaurant Name,Zip Code,Catering,Deli,Hot dogs,Mexican,Sandwich,Coffee,Vegetarian,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,0,Black Cow Burger Bar,1376,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,631.0,3798.0,74.0,29.0,1.0,18.0,293.0,346.0,268.0,449.0
1,0,Black Cow Burger Bar,1376,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,715.0,4025.0,27.0,38.0,1.0,7.0,126.0,92.0,122.0,311.0
2,0,Black Cow Burger Bar,1380,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,459.0,3319.0,21.0,17.0,0.0,16.0,55.0,60.0,70.0,213.0
3,0,Black Cow Burger Bar,1420,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,330.0,2770.0,149.0,104.0,0.0,14.0,187.0,396.0,15.0,55.0
4,0,Black Cow Burger Bar,1420,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,306.0,1844.0,41.0,126.0,0.0,6.0,132.0,271.0,24.0,179.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129506,0,Pizza Mill,99926,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,133.0,145.0,6.0,2.0,10.0,1210.0,87.0,26.0,59.0,164.0
129507,1,Pizza Mill,99927,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,181.0,1217.0,5.0,6.0,7.0,351.0,99.0,29.0,111.0,154.0
129508,0,Pizza Mill,99928,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,351.0,3013.0,9.0,33.0,4.0,186.0,239.0,103.0,58.0,116.0
129509,0,Pizza Mill,99928,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,351.0,3013.0,9.0,33.0,4.0,186.0,239.0,103.0,58.0,116.0


In [ ]:
def yesstartswith(df: dataframe, list_of_patterns: list, reverse=False: boolean) -> series of boolean values:
  """Returns 'Location' series of elements that either start with a pattern or don't start with a pattern
  
  Args:
    df - Dataframe
    list_of_patterns - List of patterns to query
    reverse - Whether to return the series of elements that start with a list of patterns or to return
    the complement of those elements
  
  Returns:
    Series of boolean values
  """
  if reverse is False:
    return df['Location'][df['Location'].apply(lambda x: \
                                                             any(map(lambda pat: x.startswith(pat), list_of_patterns)))]
  if reverse is True:
    return df['Location'][~df['Location'].apply(lambda x: \
                                                              any(map(lambda pat: x.startswith(pat), list_of_patterns)))]

In [ ]:
yesstartswith(merged_df, ['Walmart SC', 'Sam', 'Wm', 'Murphy', 'WM', 'wm'])

1352      Sam's Club
1353      Sam's Club
1354      Sam's Club
1355      Sam's Club
1356      Sam's Club
             ...    
124258    Sam's Club
124259    Sam's Club
124260    Sam's Club
124261    Sam's Club
124262    Sam's Club
Name: Location, Length: 35348, dtype: object

In [ ]:
def split_text(s):
    for k, g in groupby(s, str.isalpha):
        yield ''.join(g)

def write_csv(df):
  os.makedirs('Output_csv_files', exist_ok=True)
  distance = list(split_text(df.columns[0]))[-1]
  df.to_csv(f"Output_csv_files/merged_df_{distance}.csv")
  print("Created file: \"merged_df_{}.csv\"".format(distance))
  return print("Success!")

In [ ]:
write_csv(merged_df)